<a href="https://colab.research.google.com/github/aghadavood/LLMProjects/blob/main/DialougeSum_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What are we doing in this notebok?


1.   using FLAN-T5 model to summerizing dialouges.

2.   using ROUGE metrics to evaluate the results.


3.   Prforming PEFT (Parameter Efficient Fine-Tuning)

4.  evaluating the results after PEFT










In [2]:
# install dependeniceis

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torc

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [4]:
#load dataset and LLM
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [5]:
#load FLAN-T5, small version
#torch-dtype = torch.bfloat16 the memory type
model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name , torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
#how can pull out the number of model params and find out how many of them are trainable??

def print_number_of_trainable_model_params(model):
  trainable_model_params= 0
  all_model_params = 0
  for _,param, in model.named_parameters():
    all_model_params += param.numel()
    if param.requires_grad:
      trainable_model_params += param.numel()
  return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print (print_number_of_trainable_model_params(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable parameters: 100.00%


In [7]:
#test model with 0-shot

index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""

summarize the following conversation.

{dialogue}

summary:
"""

inputs = tokenizer(prompt , return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(inputs["input_ids"],max_new_tokens = 200,)[0],skip_special_tokens = True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:


summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

summary:

------------------------------------------------------------------

In [33]:
# Define a function to tokenize each example
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '

    # Create prompts by combining dialogue and summary
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]

    # Tokenize prompts and add tokenized inputs to examples
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids

    # Tokenize summaries and add tokenized labels to examples
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# Map the tokenization function to each batch of examples in the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns from tokenized datasets
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])

# Check the shapes of the three datasets
print(f"Shapes of datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)


Shapes of datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
})


In [40]:
import time
from transformers import Trainer, TrainingArguments

output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,  # Log training metrics every 100 steps
    evaluation_strategy="steps",  # Evaluate on the validation set every logging_steps
    eval_steps=500,  # Evaluate on the validation set every 500 steps
    save_steps=1000,  # Save model checkpoint every 1000 steps
    max_steps=-1  # Use unlimited number of training steps
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

trainer.train()


Step,Training Loss,Validation Loss
500,33.847500,35.231998
1000,30.040000,31.750000
1500,29.588800,31.266001
2000,29.491300,31.231001
2500,29.451300,31.233000
3000,29.443800,31.229000
3500,29.480000,31.233000
4000,29.330000,31.229000
4500,29.487500,31.229000


TrainOutput(global_step=4674, training_loss=30.688329054343175, metrics={'train_runtime': 3162.2739, 'train_samples_per_second': 11.821, 'train_steps_per_second': 1.478, 'total_flos': 2.559622983450624e+16, 'train_loss': 30.688329054343175, 'epoch': 3.0})

In [47]:
import os

# Define the output directory on your local machine
local_output_dir = "C:/Users/saagh/Documents/Codes/models"

# Create a timestamped directory for the model
output_dir = f'{local_output_dir}/dialogue-summary-training-{str(int(time.time()))}'

try:
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    print(f"Directory '{output_dir}' created successfully.")
except OSError as error:
    print(f"Error creating directory '{output_dir}': {error}")

# Save the fine-tuned model to your local directory
try:
    trainer.save_model(output_dir)
    print("Model saved successfully.")
except Exception as e:
    print(f"Error saving the model: {e}")

Directory 'C:/Users/saagh/Documents/Codes/models/dialogue-summary-training-1714088204' created successfully.
Model saved successfully.


In [49]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

# Define the output directory on your Google Drive
drive_output_dir = "/content/drive/My Drive/tt"

# Create a timestamped directory for the model
output_dir = f'{drive_output_dir}/dialogue-summary-training-{str(int(time.time()))}'

# Save the fine-tuned model to your Google Drive
try:
    trainer.save_model(drive_output_dir)
    print("Model saved successfully to Google Drive.")
except Exception as e:
    print(f"Error saving the model to Google Drive: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved successfully to Google Drive.
